In [1]:
from Bio import PDB
import numpy as np
from scipy.spatial.transform import Rotation
import pickle

In [2]:
parser = PDB.PDBParser()

In [3]:
with open('output/vol_to_keep.pkl', 'rb') as f:
    vol_to_keep = pickle.load(f)
kept_vol = [f'output/fit_pdb/vol_{str(idx).zfill(3)}_fit.pdb' for idx in vol_to_keep]

In [4]:
additional_vol = []
vol_to_analyze = kept_vol + additional_vol

In [5]:
beta_coords = []
beta_coords_quat = []

for num_processed, vol in enumerate(vol_to_analyze):
    vol_fit = parser.get_structure('vol_fit', vol)
    
    vol_fit_alpha = PDB.Structure.Structure('vol_fit_alpha')
    vol_fit_alpha.add(vol_fit[0]['A'])
    vol_fit_alpha.add(vol_fit[0]['C'])
    alpha_com = vol_fit_alpha.center_of_mass()
    
    vol_fit_beta = PDB.Structure.Structure('vol_fit_beta')
    vol_fit_beta.add(vol_fit[0]['E'])
    vol_fit_beta.add(vol_fit[0]['F'])
    beta_com = vol_fit_beta.center_of_mass()
    
    alpha_A_180_CA = vol_fit[0]['A'][180]['CA'].get_coord()
    alpha_C_180_CA = vol_fit[0]['C'][180]['CA'].get_coord()
    alpha_A_191_CA = vol_fit[0]['A'][191]['CA'].get_coord()
    alpha_C_191_CA = vol_fit[0]['C'][191]['CA'].get_coord()
    alpha_180_plane = np.cross(alpha_A_180_CA - alpha_com, alpha_C_180_CA - alpha_com)
    alpha_191_plane = np.cross(alpha_A_191_CA - alpha_com, alpha_C_191_CA - alpha_com)
    alpha_C2 = np.cross(alpha_180_plane, alpha_191_plane)
    alpha_C2 = alpha_C2 / np.linalg.norm(alpha_C2)
    
    beta_E_111_CA = vol_fit[0]['E'][111]['CA'].get_coord()
    beta_F_111_CA = vol_fit[0]['F'][111]['CA'].get_coord()
    beta_E_215_CA = vol_fit[0]['E'][215]['CA'].get_coord()
    beta_F_215_CA = vol_fit[0]['F'][215]['CA'].get_coord()
    beta_111_plane = np.cross(beta_E_111_CA - beta_com, beta_F_111_CA - beta_com)
    beta_215_plane = np.cross(beta_E_215_CA - beta_com, beta_F_215_CA - beta_com)
    beta_C2 = np.cross(beta_111_plane, beta_215_plane)
    beta_C2 = beta_C2 / np.linalg.norm(beta_C2)
    
    alpha_A_308_CA = vol_fit[0]['A'][308]['CA'].get_coord()
    alpha_axis_3 = np.cross(alpha_A_308_CA - alpha_com,alpha_C2)
    alpha_axis_3 = alpha_axis_3 / np.linalg.norm(alpha_axis_3)
    alpha_axis_1 = np.cross(alpha_C2, alpha_axis_3)
    alpha_axis_1 = alpha_axis_1 / np.linalg.norm(alpha_axis_1)
    
    beta_F_179_CA = vol_fit[0]['F'][179]['CA'].get_coord()
    beta_axis_3 = np.cross(beta_F_179_CA - beta_com,beta_C2)
    beta_axis_3 = beta_axis_3 / np.linalg.norm(beta_axis_3)
    beta_axis_1 = np.cross(beta_C2, beta_axis_3)
    beta_axis_1 = beta_axis_1 / np.linalg.norm(beta_axis_1)
    
    A_mat = np.hstack((alpha_axis_1[:, np.newaxis], alpha_C2[:, np.newaxis], alpha_axis_3[:, np.newaxis]))
    B_mat = np.hstack((beta_axis_1[:, np.newaxis], beta_C2[:, np.newaxis], beta_axis_3[:, np.newaxis]))
    rot_mat = np.linalg.inv(A_mat) @ B_mat
    
    r = Rotation.from_matrix(rot_mat)
    euler_angles = r.as_euler('xyz', degrees=True)
    quat = r.as_quat()
    
    alpha_A_684_CA = vol_fit[0]['A'][684]['CA'].get_coord()
    beta_E_30_CA = vol_fit[0]['E'][30]['CA'].get_coord()
    A_684_E_30_vec = beta_E_30_CA - alpha_A_684_CA
    alpha_beta_com_vec = beta_com - alpha_com
    beta_coords.append(np.hstack((np.linalg.inv(A_mat) @ A_684_E_30_vec, euler_angles)))
    beta_coords_quat.append(np.hstack((np.linalg.inv(A_mat) @ alpha_beta_com_vec, quat)))
    
    if num_processed % 100 == 0: print(f'{num_processed} / {len(vol_to_keep)} processed')
    if num_processed == len(vol_to_keep) - 1: print('finished processing')

0 / 862 processed
100 / 862 processed
200 / 862 processed
300 / 862 processed
400 / 862 processed
500 / 862 processed
600 / 862 processed
700 / 862 processed
800 / 862 processed
finished processing


In [6]:
beta_coords = np.array(beta_coords)
beta_coords_quat = np.array(beta_coords_quat)

In [7]:
# beta coords contains translation between PCET residues and rotation as euler angles
with open('output/turnover_beta_coords.pkl', 'wb') as f:
    pickle.dump(beta_coords, f)

In [8]:
# beta coords quat contains translation between alpha and beta COMs and rotation as quaternions
with open('output/turnover_beta_coords_quat.pkl', 'wb') as f:
    pickle.dump(beta_coords_quat, f)